# 전력 수요 예측 프로젝트 - 모델별 분석
- 김도영, 김남훈  
- 모든 분석은 **시계열데이터분석_3~5주차.pdf** 내용을 기반으로 구성하였습니다.


In [ ]:
# -- 데이터 불러오기 및 전처리 --
# 공통: 시계열데이터분석_3주차.pdf 4쪽 참고 (시간형식 처리)

import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# 데이터 로딩
df = pd.read_csv("한국전력거래소_시간별 전국 전력수요량_20241231.csv", encoding='cp949')
df = df.melt(id_vars='날짜', var_name='시간', value_name='수요량(MWh)')
df['시간'] = df['시간'].str.replace('시','').astype(int)
df['일시'] = pd.to_datetime(df['날짜']) + pd.to_timedelta(df['시간'] % 24, unit='h')
df.loc[df['시간'] == 24, '일시'] += pd.Timedelta(days=1)
df = df[['일시','수요량(MWh)']].sort_values('일시').set_index('일시')
df.head()


## ⏱️ 시간 단위 예측 (ARIMA)
- **출처:** 시계열데이터분석_4주차.pdf 8쪽  
- ARIMA(p,d,q) 모델을 사용하여 단변량 시계열을 예측합니다.


In [ ]:
# train/test 분리
split_idx = int(len(df)*0.8)
train = df.iloc[:split_idx]
test = df.iloc[split_idx:]

# 모델 학습 및 예측
model = ARIMA(train, order=(3,1,2))
model_fit = model.fit()
pred = model_fit.forecast(steps=len(test))

# 성능 평가
mae = mean_absolute_error(test, pred)
rmse = mean_squared_error(test, pred, squared=False)
print(f'ARIMA 시간단위 예측 → MAE: {mae:.2f}, RMSE: {rmse:.2f}')

# 시각화
plt.figure(figsize=(15,5))
plt.plot(train.index, train['수요량(MWh)'], label='Train')
plt.plot(test.index, test['수요량(MWh)'], label='Test')
plt.plot(test.index, pred, label='ARIMA 예측')
plt.title('ARIMA - 시간 단위 예측')
plt.legend()
plt.show()


## 🌸 계절 단위 예측 (SARIMA)
- **출처:** 시계열데이터분석_3주차.pdf 16~18쪽  
- 계절성 반영을 위해 SARIMA(p,d,q)(P,D,Q,s) 모델 사용


In [ ]:
# 분기별(3개월) 리샘플링
df_q = df.resample('Q').sum()
train_q = df_q.iloc[:-2]
test_q = df_q.iloc[-2:]

model = SARIMAX(train_q, order=(1,1,1), seasonal_order=(1,1,1,4))
model_fit = model.fit()
pred = model_fit.forecast(len(test_q))

mae = mean_absolute_error(test_q, pred)
rmse = mean_squared_error(test_q, pred, squared=False)
print(f'SARIMA 계절 예측 → MAE: {mae:.2f}, RMSE: {rmse:.2f}')

# 시각화
plt.figure(figsize=(12,5))
plt.plot(train_q, label='Train')
plt.plot(test_q, label='Test')
plt.plot(pred, label='SARIMA 예측')
plt.title('SARIMA - 계절(3개월) 단위 예측')
plt.legend()
plt.show()


## 📆 연도 단위 예측 (ARIMA)
- **출처:** 시계열데이터분석_5주차.pdf 5~7쪽  
- 연도별 추세를 보기 위해 연 단위 집계 및 ARIMA 적용


In [ ]:
df_y = df.resample('Y').sum()
train_y = df_y.iloc[:-1]
test_y = df_y.iloc[-1:]

model = ARIMA(train_y, order=(1,1,1))
model_fit = model.fit()
pred = model_fit.forecast(len(test_y))

mae = mean_absolute_error(test_y, pred)
rmse = mean_squared_error(test_y, pred, squared=False)
print(f'ARIMA 연도 예측 → MAE: {mae:.2f}, RMSE: {rmse:.2f}')

# 시각화
plt.figure(figsize=(8,4))
plt.plot(train_y, label='Train')
plt.plot(test_y, label='Test')
plt.plot(pred, label='ARIMA 예측')
plt.title('ARIMA - 연도 단위 예측')
plt.legend()
plt.show()
